<a href="https://colab.research.google.com/github/jawshan/CompNeuro_Summer2020/blob/master/Jawshan_summer_2020_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
#@title Data retrival
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


In [81]:
#@title Import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [82]:
#@title Data loading
import numpy as np
import pandas as pd

alldat = np.array([])
for j in range(len(fname)):
    alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))
dat = alldat[11]

In [83]:
#@title Data Extraction
def extract_data(data, brain_area):
  row = {}
  row['mouse_name']     = data['mouse_name']
  row['num_v1_neurons'] = np.sum(data['brain_area']=='VISp')
  row['num_neurons'], row['num_trials'], row['num_time_bins'] = data['spks'].shape
  row['start_time']      = 0.0
  row['stim_onset_time'] = data['stim_onset']
  row['gocue_time']      = data['gocue']
  row['response_time']   = data['response_time']
  row['feedback_time']   = data['feedback_time']
  row['end_time']        = data['bin_size'] * row['num_time_bins']
  row['contrast_right'] = data['contrast_right']
  row['contrast_left']  = data['contrast_left']
  # feedback (-ve,+ve)          :  ∈ {-1,1}  
  row['response']        = data['response']
  row['feedback']        = data['feedback_type']
  row['binary_feedback'] = np.array(data['feedback_type'] == 1).astype(int)
  ## Extracting spikes from neurons corresponding to specified region
  visp_indexes = np.argwhere(data['brain_area']==brain_area)
  reduced_neuron_spikes = np.squeeze(data['spks'][visp_indexes,:,:],1)  
  ## Reshaping spikes to have trials dimension first : #trials, #neurons, #time_bins 
  row['spikes'] = reduced_neuron_spikes.transpose(1,0,2)
  ## Convert spikes to time & spike count per trial
  row['spike_times'] = np.zeros_like(row['spikes']).astype(np.float16)
  ax1, ax2, ax3      = np.shape(row['spikes'])
  row['spike_count'] = np.zeros((ax1, ax2, 1)).astype(np.float16)
  correction         = 1/1000
  for i, trial in enumerate(row['spike_times']):
    for j, neuron in enumerate(row['spike_times'][i]):
      row['spike_count'][i][j] = np.sum(row['spikes'][i][j])
      spike_time_list          = [10 * bin if spike !=0 else 0 for bin, spike in enumerate(row['spikes'][i][j])]
      row['spike_times'][i][j] = np.array(spike_time_list).astype(np.float16)
  row['spikes_reordered'] = reduced_neuron_spikes
  return(row)
extracted_data = np.array(list(map(lambda data: extract_data(data, 'VISp'), alldat)))


In [17]:
#@title Data Table
time_cols = ['start_time','stim_onset_time','gocue_time','response_time','feedback_time','end_time']
id_cols  = ['mouse_name']
dim_cols = ['num_v1_neurons','num_neurons','num_trials','num_time_bins','spike_times']

cols = id_cols + dim_cols

df = pd.DataFrame.from_records(extracted_data, columns=cols)

df_v1 = df[df['num_v1_neurons'] > 0]
df_v1

,mouse_name,num_v1_neurons,num_neurons,num_trials,num_time_bins,spike_times
0,Cori,178,734,214,250,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 50.0, 0.0, 0.0, 0...."
2,Cori,114,619,228,250,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,Forssmann,39,1769,249,250,"[[[0.0, 10.0, 0.0, 0.0, 40.0, 0.0, 0.0, 0.0, 0..."
7,Hench,48,1156,250,250,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
9,Hench,105,1172,447,250,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
11,Lederberg,66,698,340,250,"[[[0.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
13,Lederberg,42,756,268,250,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
19,Moniz,122,899,235,250,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
21,Muller,133,646,444,250,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
24,Radnitz,94,885,261,250,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [68]:
#@title Dif model -- achieves class likelihood
from sklearn.linear_model import LogisticRegression

my_mouse             = extracted_data[0]
feedback             = my_mouse['binary_feedback']
spikes_my_mouse      = my_mouse['spikes_reordered']
neurons, __, __,     = np.shape(spikes_my_mouse)
neurons              = [neuron for neuron in range(neurons)]

## assuming L2 > L1 as vector already sparse

all_accuracies  = []
neuron2accuracy = {}

for neuron in neurons:
  X         = spikes_my_mouse[neuron][1:][:]
  y         = feedback[:-1]
  

  """ Balance classifiers
  """
  zeros     = len(y[y == 0])
  mask      = np.argsort(y)
  slice_arr = mask[:(2 * zeros)]
  slice_arr = np.sort(slice_arr)
  X         = X[slice_arr]
  y         = y[slice_arr]
  """ End balance classifiers
  """


In [88]:
#@ logistic regression
#from sklearn.linear_model import LogisticRegression

my_mouse             = extracted_data[11]
feedback             = my_mouse['binary_feedback']
spikes_my_mouse      = my_mouse['spikes_reordered']
#neurons, trials, bins, = np.shape(spikes_my_mouse)
neurons, trials, bins, = np.shape(spikes_my_mouse[:,:,50:150])

LDR = np.zeros([trials, neurons])
for t in range(trials):
    for n in range(neurons):
      spike_counts= sum(spikes_my_mouse[n][t])
      LDR[t,n] = spike_counts

#split training-test data
num_trials_in_training = int(np.round(trials * 0.8))
training_data = LDR[range(num_trials_in_training), :]
test_data = LDR[range(num_trials_in_training, trials), : ]
training_feedback = feedback[range(num_trials_in_training)]
test_feedback = feedback[range(num_trials_in_training, trials)]

# define model
log_reg = LogisticRegression(penalty="none")

# fit model to data
log_reg.fit(training_data, training_feedback)
#test data
def compute_accuracy(X, y, model):
  y_pred = model.predict(X)
  accuracy = (y == y_pred).mean()
  return accuracy

test_accuracy = compute_accuracy(test_data, test_feedback, log_reg)
print(f"Accuracy on the testing data: {test_accuracy:.2%}")

#cross validation accuracy
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), LDR, feedback, cv=5)
if not np.isnan(accuracies).any():
  mean_accuracy           = accuracies.mean()
  print(f"Cross-validated Accuracy: {mean_accuracy:.2%}")
  

Accuracy on the testing data: 72.06%
Cross-validated Accuracy: 60.88%


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
